## Maximum Independent Set

Write a function `max_independent_set(G, T, B)` which computes a maximum independent set of a graph `G` with a nice tree decomposition `(T, B)`. Algorithm should perform well on graphs with small tree width.

In [3]:
def bucket_elimination(G):
    T = Graph()
    B = {}
    vrt = sorted(G.vertices(sort=False), key=G.degree, reverse=True)
    T.add_vertices(vrt)
    for x in vrt:
        B[x] = set([x])
    
    for x, y in G.edges(labels=False, sort=False):
        if vrt.index(x) < vrt.index(y):
            B[y] = B[y] | set([x])
        else:
            B[x] = B[x] | set([y])
    
    for x in reversed(vrt):
        A = copy(B[x])
        A.remove(x)
        if len(A) > 0:
            y = max(A, key=lambda z: vrt.index(z))
            T.add_edge((x, y))
            B[y] = B[y] | A
    
    return T, B

def DDFS(T, r):
    """Directs the tree T to the root r."""
    active = [r]
    prev = {}
    while len(active) > 0:
        v = active.pop()
        for w in T.neighbors(v):
            if w not in prev and w not in active and w != r:
                prev[w] = v
                active.append(w)
    DT = DiGraph()
    DT.add_edges(prev.items())
    return DT


def nice_tree_decomposition(G):
    T, B = bucket_elimination(G)
    ntd_handle_leaves(T, B)
    ntd_handle_edges(T, B)
    r = next(v for v in T.vertices(sort=False) if len(B[v]) == 1)
    DT = DDFS(T, r)
    ntd_handle_multiple_children(DT, B)
    return DT, r, B

def new_vertex(G):
    """Returns integer v such that v, v + 1, v + 2, ... can be used as new vertices in G."""
    vrt = [0] + [x for x in G.vertices(sort=False) if type(x) == type(1) or type(x) == type(int(1))]
    return max(vrt) + 1

def ntd_handle_leaves(T, B):
    leaves = [x for x in T.vertices(sort=False) if T.degree(x) == 1]
    nv = new_vertex(T)
    
    for l in leaves:
        A = copy(B[l])
        while len(A) > 1:
            T.add_edge((l,nv))
            A.pop()
            B[nv] = copy(A)
            l = nv
            nv = nv + 1
            
def ntd_handle_edges(T, B):
    nv = new_vertex(T)
    
    for (x, y) in T.edges(labels=False, sort=False):
        Bx = copy(B[x])
        By = copy(B[y])
        Bxy = Bx & By
        
        if len(By) < len(Bx):
            x, y = y, x
            Bx, By = By, Bx
        
        T.delete_edge((x, y))
        
        path = [a for a in Bx if a not in Bxy]
        while path != []:
            a = path.pop()
            T.add_edge((x, nv))
            Bx.remove(a)
            B[nv] = copy(Bx)
            x = nv
            nv = nv + 1
        
        path = [a for a in By if a not in Bxy]
        path.pop()
        while path != []:
            a = path.pop()
            T.add_edge((x, nv))
            Bx = Bx | set([a])
            B[nv] = copy(Bx)
            x = nv
            nv = nv + 1
        T.add_edge((x, y))
        
def ntd_handle_multiple_children(DT, B):
    big_vertices = [x for x in DT.vertices(sort=False) if DT.in_degree(x) > 2]
    nv = new_vertex(DT)
    
    while big_vertices != []:
        v = big_vertices.pop()
        Nv = DT.neighbors_in(v)
        Nv.pop()
        for u in Nv:
            DT.delete_edge((u, v))
            DT.add_edge((u, nv))
        DT.add_edge((nv, v))
        B[nv] = copy(B[v])
        if len(Nv) > 2:
            big_vertices.append(nv)
        nv = nv + 1
    
    big_vertices = [x for x in DT.vertices(sort=False) if DT.in_degree(x) == 2]
    for v in big_vertices:
        u,w = DT.neighbors_in(v)
        if B[u] != B[v]:
            DT.delete_edge((u, v))
            DT.add_path((u, nv, v))
            B[nv] = copy(B[v])
            nv = nv + 1
        if B[w] != B[v]:
            DT.delete_edge((w, v))
            DT.add_path((w, nv, v))
            B[nv] = copy(B[v])
            nv = nv + 1

def decomposition_width(B):
    return max(len(B[x]) for x in B) - 1


In [32]:
def is_tree_decomposition(G, T, B):
    bags = list(B.values())
    for v in G.vertices(sort=False):
        has_bag = False
        for b in bags:
            if v in b:
                has_bag = True
                break
        if not has_bag:
            return False
    for u, v in G.edges(labels=False, sort=False):
        has_bag = False
        for b in bags:
            if v in b and u in b:
                has_bag = True
                break
        if not has_bag:
            return False
    for v in G.vertices(sort=False):
        tverts = []
        for tv, b in B.items():
            if v in b:
                tverts.append(tv)
        TS = T.subgraph(tverts)
        if not TS.is_connected():
            return False
    return True

def is_nice_tree_decomposition(G, T, B):
    if not is_tree_decomposition(G, T, B):
        return False
    for v in T.vertices(sort=False):
        nin = T.neighbors_in(v)
        if len(nin) == 0: # leaf
            if len(B[v]) != 1:
                print(f"leaf {v} has bag of size {len(B[v])}")
                return False
        elif len(nin) > 2: 
            print(f"vertex {v} has 3 or more children")
        elif len(nin) == 1:
            u = nin[0]
            ints = B[v] & B[u]
            if len(B[v] - ints) > 1 or len(B[u] - ints) > 1:
                print(f"verices {v} and {u} have bags {B[v]} and {B[u]} with difference > 1")
        # len(nin) == 2
        elif B[v] != B[nin[0]] or B[v] != B[nin[1]]:
            print(f"children of {v} have different bags")
            return False
    return True

In [4]:
def is_independent_set(G, X):
    return G.subgraph(X).num_edges() == 0

In [195]:

def max_independent_set(G, DT, r, B):
    """Returns maximal independent set of graph G with nice tree decomposition DT, r, B (directed tree with root r and 
    bucket map B. we also assume that len(B[r]) = 1). 
    We can obtain DT, B and r using nice_tree_decomposition function above"""
    MEM = {}  # Memorize all results of max_independent_set_rec
    IS1 = # ......
    IS2 = # ......
    return IS1 if len(IS1) >= len(IS2) else IS2

def max_independent_set_rec(G, DT, B, t, S, MEM):
    """Returns the largest independent set of the subgraph of G induced on bags below (including) t 
    which contains all vertices from S, S subset of B[t] (More precisely, vertices of this independent set from B[t] are 
    exactly vertices S).
    Parameters: 
      G        graph
      (DT, B)  tree decomposition with tree directed towards the root
      t        tree vertex
      S        a independent set of bag B[t]"""
    if (t, tuple(S)) in MEM:
        return MEM[(t, tuple(S))]  # If we already calculated the result
    result = None
    sons = DT.neighbors_in(t)
    # ......
    # ......
    # ......
    # ......
    MEM[(t, tuple(S))] = result  # Memorize the result
    return result

### Tests

In [196]:
G1 = Graph('XTnNw?DOYHgJ@BP@g`wG^PAoa?@C?G??Ga?EG_@oC?NcO?}???P')
T1, r1, B1 = nice_tree_decomposition(G1)
MI1 = max_independent_set(G1, T1, r1, B1)
(is_independent_set(G1, MI1), len(MI1)) == (True, 7)

True

In [197]:
G2 = Graph('~?@tF^Z|r]]S?W@o@OB_@}?ZoBp?Bm?_E?_F??BOC?M??B_?_????@?@G@_?c?x?_??GC??C?o??@@_??@@_??K_O??FGG???g?@?C???C?O@??G?_B??G?_B_?C?O?g???C?Z??G?_B[???A?HXG_????@G_????CcO????B@C?????@A??????CG??????GO?????@???????Ac???????w???????M?@??G????????_???G?G?B????O?G?B????S???@????H????W????????A?????????C????B?????????COAo????????????@`?????????OO??@????_@????D?@O?????????A??????????A_???????K?@O???????FO?S????????i?A_???????D_???@???????G???????????G???A??????@c???@???????w??????O???_???????A???C?C??????G???O?W??????O???_?g???????L????????????Eo????????????C????W???????L_???B_???????E????J???????@k????V????????????????I???????????G?A???????????B??S???????????M??B??????????????A???????????K??A???????????H?B???????????????????????????@???C????????????_??O???????????a???_???????????F??????????????@F_??O???????????`w??C???????????G\\?C?????O?????????_O?????_????????H??????@?????????Q?C??????????????@?A??????????????@O????????????????G????????????????B_?O??????????????N??_??????????????^O????????????????Z_C???G????????????????????????????????G???O????????????K?_??@?????????????W?????C??????????????????G??????????????????C????????????B')
T2, r2, B2 = nice_tree_decomposition(G2)
MI2 = max_independent_set(G2, T2, r2, B2)
(is_independent_set(G2, MI2), len(MI2)) == (True, 44)

True

In [198]:
G3 = Graph('~?Bnn^~dkZH@Gc@GE@E_a@?@HCKcO`??xG`{COf???G??`_?AA??CJ??CN_?AEGG`??@CG???O_?GG`??wG@??{??_?^@CG?Cw?@??a?????O?????`?C???C_A???Qw?_??C`aG?????G?????KP?????SP?????UG_????CaG????Bo??@o??????F???@@????Q??@???CQ???_??A@??W????AO?D????CQ??{???????@oG?I?????????_???W????G???G@???C???B?H??@?????????C?????@G?AG?????BG?AG?????Bc?@??????@p?????????J??_?????????Q????????G?S????????W?s????????[?Y????????JG??G??????????@??????@?C??C??????C@G?????????GE???G???????????E????????G??@_???????B????????????K???_???????@w??@_????????w??@_???????@{???o???????@v???G????????????@?????????C_??C????????AQ???G?????????d???G?????????e_???????????ABwG@???O????????@????Q?????????????@G????????WG@??A?????????gG???A?????????s??????????????D@?????????????@WG?????????????b__????????????Am@?????????????@\\??????????G???????????????C???????????????@_????A??A???????K?????K??????????O?????W??????????????U????????????????U?C??????????????J?B???????????????o?O??????????_????A?????????????????G@?????????????????A??G?????????????????C???????????????G?@???O???????????A??G??A????????????C????????????__?????????????????B?????????????????@B?????K????????????`_????F???????????????K_?????????????????@c???_??????????????M???B???????????????O_??C???????????????[_??D_??????@_A????????????????????_?????????????????@????????????K???????K?O???????????????????????????_???????????????????A_???C????????????????O???A????????????????S????G???????????????A_????_???????????????I????C?????????????????????H????????????????O????M_???????????????A????Fw???????????????A?????}?@??????????????????????L????????????????????G?Y??????????????????????W????????????????????C?L????????????????????K?AO???????????????????Bw????A???????????????????????G??A??????????????????????????????????????A?????O??C??????????????A_????G?????????????????@w????????_??????????????Z????????C??????????????@o????@???O??????????????Io??????????????BC???????????????????????A????????????????????????@q????????@????????????????P????????????????????????A?????????_????????????????????????B????????????????P????????F????????????????O????????B_??@???@??????????????????????O???O?_?????????????????????O??G??????????????????????@???_?????????????????G????A??@??????????????????W???????@??????????????????K????@???_?????????????????N?????O??G?????????????????@G????A??@??????????????????F_????G????????????????????@X???????????????????????????C???????????????????????????_???????????????????????????OG???????????????????????????b???????????????????????????_{??????????????????????????ARW??????????????????????????CEo??????????????????????????Cf{???_?????@O?????????????????????G??????O??????????????????G???????G?????????????????????????????_????????????????????@???????@?C???????????????????B???????@?C???????????????????B?????????????????????????????????????????????????????????A??@??????????????????????????????C??????????????????????????H???o?????????_????????????????????????????O??????????????????????C???O???K?O????????????????????@???C???B???????????????????????g?????`???????????????????????????????C????????????????????????@?C????G????????????????????????B????????e???????????????????????????????R???????????????????????????????CO??????????????????????BW???????????????????????????????@_???????????????????????????????J????????????????????????????????W????????????????????????????????[_???????????????????????????????JX???????????????????????????????@pG???????????????????????????????^_AC????????????????????????????????CG??????????????????????????????@??G??????????????????????????????@_AC???????????????????????????????O?????????????????????????????????N?CG??????????????????????????????@w?O_??????????????????????????????Aw?`???????????????????????????????Kw???_??????????????????????????@??????????????????????????????????????????????????????????????????????G???G?????@?????????????????????????C??????????????????????????????????????@??????G????????????????????????????B???????????????????????????????????@_?????C????????????????????????????@G?????@?????????????????????????C???G??????G????????????????????????????B{???O???????????????????????????_???_????_????????????????????????????????G???_??????????????????????????@????_G?@?????????????????????????????????????O????????????????????????????????????A??????????????????????????????????@_?G??????????????????????????????????E?@???????????????????????????????????I?????????????????????????????????????L_?g??????????????????????????????????Fw?I???????????????????????????????????~???????????????????????????????????????????????????????????????????U?????????G????????????????????????????K?????????W????????????????????????????k?????????S????????????????????????????S?????????C????????????????????????????@?????????Aw????????????????????????????C?????????^_')
T3, r3, B3 = nice_tree_decomposition(G3)
MI3 = max_independent_set(G3, T3, r3, B3)
(is_independent_set(G3, MI3), len(MI3)) == (True, 85)

True

In [188]:
import time
start_time = time.time()
G4 = Graph('~?FYZk[v|^NpaUH?BkP@e??_BKKGowX`wW`sC??DO?KG??_A?G?C?p?C?`_A?Og?_EN?C?p{??@??O???GO?A?KG???FA??O@o??A?FC?????W?????W?????k?????X?????LW????@[_????Bz?????Ix?????A?_????@@C???????a??????aGg????@????????@???????A_??????GG??????F@???????gS???????Wg??????Egg??????FWO????????C??I?????G_?@O????@????O?????????g?????????g?????????O?????w?????????EG??????????_?????S???G??????G???W??????G???K_?????C???M??????C???Bo?????H_??????????e????@?????@K????B??????K????A??????e????@w?????G?????N???C?????????O??O?????????A??_????????Wa??_????????[O???????????NCO?C????????AG??????????????????a???????@????@C???????B?????C????????_????a???????@w????G_???????^????@C???????AO????CO???????FWI?@O???????????G??O??????????C????_????D?????????G?????O??????????????I????@_???C??????????FC???????????????C???????????????_@???????????????_O?????????????F????????????????SOG?????????????Bw?O?????????????Fg?O?????????????DSK?G??????????????B????????????????GG?O?????????????@@_???????????????FA????????????????NA????????????????C`_@??????????????FgG?@O?????????????????G???????????????C?????????????A???W??@O??????????C???wG?@O??????????C???[C??_??????????????Z??????????????D?????????G?????????g????_??????????g??????????????????@O???????@??????????@????????@_??????????g????????w??????????A????????B??????????@O???????@[??????????D????????Fw??????????I????????BoG????????????????????D??G????????????????@?O?A??????????????????A???????????????????B__?@?????????????????JC??G??????????????????C??A??????????????????a??B??????????????????W_??_?????????????????F???E??????????????????{O??g?????????????????A?????G?????????W????????????G?????????G???????C????C?????????[???????B????@?????????F????????g??????????????????H???????????????????????s??????????????????????@g???B???G?????E?????????????????D????????????????????G??@???????????????O????????G??????????????B?????C???_??????????????K?????WG?????????????????W?????OG?????????????????G?????[G????????????????c???????A????????????????G???????????O?????????????????W??????@?????????????????D_??????A?????????????????@???@???A?????????????????H???@???@?????????????????D_??@OG?????????????????????????@????????????????????C????CC????????????????????O????G??????????????????????????oG????????????????????_????s??????????????????????????R??????????????????????????DoG????????????????????_????j??????????????????????????????????G????A???????????????@??????G????A???????????????@_?????C?????????????????????W?A?????????????????????????????????C??????????????????????@?????O????????????????????O????????????????????????????w????????????????????????????G???????O????????????????????^???I????????????????????????????@O?????O???????????????????_??@?????@???????????????????B?????????G???????????????????????I?????????????????????????????D?????C????????????????????O??????????????????????????_O????????????????????????????CI??C??????????????????????????O???O??????????????????????????`O??w??????????????????????????`O??s??????????????????????????????N??????????????????????????CI??FO????G????????G???????????????????????????????_????@???????????@C???????????????????????????????C???????????????????????????????_????????????C??????????????????W_??????????????????????????????FC????????????G??????????????????kO????????????????????B???????????_??O?????????????????E??????????G_??O?????????????????E??????????GO??G?????????????????B??????????FC??A??????????????????_?????????@w??????G????????????????????????????????G_????????????????????????????????P??????????????????????????B??????@??????????????????????????B??????G??????????????????????????@w?????A???????????????????????????\\??????@??????????????????????????@s????????????????????G????????????????????????????????????????????????G???????????????????CO?????????????G???????????????????AG?????????????E?O??????????????????????????????????A???????????????????????????????????I???????????????????????????????????S?????????????????????????????????@?S?????????????????????????????????????????????????????????????????????Bw???????????????????????????????????l?S?????????????????????????????????Fw??????????????????G?????P????????????????????????????????????a????????????????????????????????????_???????????K????????????????????????@???????????F?@O???????????????????????????????????O???????????????@????????????????????@????????????????C???????A???????????@?????????????????????????C???????????BA????????????????????????C???????????B@????????????????C???????????????????@gO???????????????@????????????????????^A????????????????G???????C???????????B{???????????????????????????????????????????????????@???????????????????O??????????????????@???????????????????O?????G?????????????_??????????????????G?????E?????????????G??????????????????A?????@O?????????????????????@W??????????????????????????????????????D????????????????@???A??????????????????B????????????????B?????????E???????????????????????????????????????B??????????????G???????????????G?????????O?????????????????????????????A????????????????????????????????????????[?????????O?????????????????????????????@w?????????o?????????????A????????????????o???????_?????????????????????????????????????????????????????????????O????????????????????????????????????????C????????????G???????_????????????????????_???????????A????????????????????????????A????????????J?????????????????????????????????????????\\???O?G????????????????????????????????????@????C????????????????????????????????????G??A?@????????????????????????????????????BA??O?G????????????????????????????????????KG?@??_????????????????????????????????????GO?A??????????????????????????????????????@o_???????????G??????????????????????????????O???????????C??????????????????????????????C???????????@??????????????????????????????W_?????????????????????????????????????????AA????????????_?????????????????????????????MC??????????????????????????????????????????W????????????@??????????????????????????????^_??????????????????????????????????????????LO??????????????????????O????????????????K??????????????????????????A????????????????@_??????????????????????????G????????????????A????G?????????????????????A??????????@?????????????????????????????????A??????????????????????C????????????????????????????????????????????BG?????A?????????????????????????????????????????????O??????????????????????????????????????c?????@???????????????????????????????????????L?????????????????????????????????????????????H?????????????????????????????????????????????__????????????????????????????????????????????W?????????????????????????????????????????????FH?????????????????????????????????????????????[_????????????????????????????????????????????Ax_????????????????????????????????????????????Do????????????????????????????e??????????????????????????????????????????????Q?????????????????@???C????????????????????????????????????????????????????????@??????????????????????????????????A????????????C?????????????????????????????????B???C?????????G?????????????????????????????????FC????????????G?????????????????????????????????B_???????????????????????????AG?????????????????K?????????????????????????????_?????????????????B?????????????????????????????CO?????????????????GC????????????????????????????@??????????????????_g????????????????????????????a???????????????????W????????????????????????????a?????????????????A@OK????????????????A???????????????????????????????A????????????????@_??????????????????????????????GW????????????????C??????????????????????????????@`_????????????????o??????????????????????????????FB????????????????@???????????????????????????????HA????????????????????????G????????????????????????@?????????????????????????????????????????????????GO???????????????????????@????????????????????????@?????????????????????????G????????????????????????SG????????????????????????_???????????????????????@wO???????????????????????@????????????????????????Bw?????????o??????????????????????????????????????????????????G?????????????@????????????????????????????????????E??????????????O???????????????A??????????W?????????o?????????????A??????????????????????????B_????????B??????????????G???????????????@??????????N???E???????????????????????????????????????????????????E????????????????????????????????????????????????_??B????????????????????????????????????????????????O???O???????????????????????????????????????????????E???E????????????????????????????????????????????????w???W???????????????????????????????????????????????B_???O???????????????????????????????????????????????Fw???o???????????????????????????????????????????????F{_?????G??????????????????????????????????????G???????G?????A??????????????????????????????????????????????G?????????????????????????????????????????????O??????@??????@??????????????????????????????????????@???????C???O?Q???????????????????????????????????????????????????O?O????????????????????????????????????????????????_??G?H?????????????????????????????????????????????????O?A?A????????????????????????????????????????????????E?G?O????????????????????????????????????????????????????_O???????????????????????????????????????????????????G@?a???????????????????????????????????????????????????O??A???????????????????????????????????????????????????[?_@???????????????????????????????????????????????????F??????A_?????????????????????????????????????????????????????CS????????????????????????????????????????????????_??????O???????????????????????????????????????????????A??????a_???????????????????????????????????????????????F??????????????????A????????????????????????????????????????A????????????????????????????????????????????????C???????????????????????????????????????????????????????????@???C????????????????????????????????????????????????G???G???O????????????????????????????????????????????????_???g????????????????C????????????????????????????????????????????????????????k???????????????????????????????????????C????????????????S???????????????????????????????????????A????????????????@????????????????????????????????????????O????????????????k???????????????????????????????????????B_???????????????A_???????????????????????????????????????U????????????????D_???????????????????????????????????????X??????G?????o??????????????????????????????????????????????????????B??????????????????????????????????????????????????????????o???????????????????????????????????????????????@?????????C????????????????????????????????????????????????C?????????O????????????????????????????????????????????????w?????????o???????????????????????????????????????????????@G?????????_????????????????????????????????????????????????s?????????W????????????????????????????????????????????????\\?????????A????????????????????????????????????????????????Fw????????????????????????????????????????????????a??????????????????????????????????????????????????????????CG?????????@@???????__??????????????????????????????????????????????????????????__??????????????????????????????????????????????????__??????O???????????????????????????????????????????????????G?????A????????????????????????????????????????????????????????????OO????????????????????????????????????????????????????C?????@@????????????????????????????????????????????????????????????A?????????????????????????????????????????????????????G?????AA?????????????????????????????????????????????????????{??????@?????????????????????????????????????????????????????V????????????????????????????????????????????????????????????@W??????????A??????????????????????????????????????????????????????????????????????g????????????????????????????????????????@????????????????????O????????????????????????????????????????B????????????????????O????????????????????????????????????????B???????????G????????G????????????????????????????????????????@w???????????????????I?????????????????????????????????????????\\?????????????????????????????????????????????????????????????Bk????_??????????w??????????????????????????????????????????????????@??????????@???????????????????????????????????????????????G??????????????@_??????????????????????????????????????????????K????_??????????O??????????????????????????????????????????????E?????????????????????A???????????????????????????????????????????????????????????????O???A?????????????????????????????????????_????????????????????@????G????????????????????????????????????@?????????????????????????K???????????????????????????????????????????????????????????????K??????????????????????????????????????_????????????????????????A??????????????????????????????????????G????????????????????????@_?????????????????????????????????????B????????????????????????????????????????????????????????????????G??????????????????????????????????????????????????o???????????????????????????????????????????????????????????????@??????????????G?????????????????????????????????????????????????@_?????????????K??????????????????????????????????????????????????o?????????????B??????????????????????????????????????????????????K?????????????@w?????????_?????????????????????????????A_?????????????????????????????????A??????????????G???????????????A????????????????????????@?????????C??????????????????????????????O????????????????????????@??????????????????C?????????????????????????????????????????????????????????????????B_??????????????????????????????????????????????GG??O??????????????I???????????????????????????????????????????????@??????????????????O?????????????????????????????????????????????????????????????C??????????????????????????????????????????????????????????????????????????D???????????????????????????????????????????G??????????????G???????C???????????????????????????????????????????G?????????????????????????????????????????????S??????????????????????????????????????????????????????????????????D?????????????????????G?????????????????????????????????????????????g????????????????????@_????????????????????????????????????????????A_????????????????????D?????????????????????????????????????????????D?????????????????????F?????????????????????????????????????????????C?????????????????????J_?????????????????????????????????????????????A????????????????????????????????????????????????????????????????????_????????????????????@@?????????????????????????????????????????????C?????????????????????K??????????????????????????????????????????????O?????????????????????wG?????????????????????????????????????????????_????????????????????@wG?????????????????????????????????????????????_????????????????????@{C?????????????????????????????????????????????O?????????????????????s??????????????????????????C?????????????????????????????????????????????????????????????????????O??????????????????????????????????????????_?????????????????????????B??????????????????????????????????????????BC???????????????????????????????????????????????????????????????????????????????????????????????????????????????????_???????????????????????a?????????????????????????????????????????????O???????????????????????W_????????????????????????????????????????????????????????????????????D@???????????????????????F???????????????????????????????????????????????????_?????????????????????????????????????????????????????????????????????H?????????????????????????????????????????????????????????????????G??A?H?????????????????????????????????????????????????????????????????K????C_????????????????????????????????????????????????????????????????B???O?G?????????????????????????????????????????????????????????????????g??A???????????????????????????????????????????????????????????????????@_????c?????????????????????????????????????????????????????????????????F????@A???????????????????????????????????????????????????????????????????????E??????????????????????????????????????????????????????????????????_?????????????A?????????????????????????????????????????????????????????????????C????????????????????????????????????????????????????????????????????????_???????????????????????????????????????????????????_???????????C???????A????????????????????????????????????????????????????????????????G???????C???????????????????????????????????????????????????C????????????????O??????????C???O????????????????????????????????????A????????????????????G??????????A???G????????????????????????????????????@????????????????@???A????????????????????????????????????????????????????O????????????????????O??????????C???O????????????????????????????????????A????????????????B_??@???????????O??@?????????????????????????????????????G????????????????C????????????????????????????????????????????????????????????????????????????????????????????????A?????????????????_???A???????????????????????????????????????????????????@?????????????????O???@????????????????????????????????????????????????????O????????????????C????O?????????????????????????????????????????????????????????????????????_???A?????????????????????????????{?O??O?????????????????????????????????????????????????????????????????????????C_?????????????????????????????????????????????????????????????????????G???C_?????????????????????????????????????????????????????????????????????????AO?????????????????????????????????????????????????????????????????????F?C??c?????????????????????????????????????????????????????????????????????@O?_?C_?????????????????????????????????????????????????????????????????????N_A??A??????????????????????????????????????????????????????????????????????m???C?????????O???????????????????????????????????????????????????????????????????????????O?????????????????????????????????????????????????????????????????A????????????????????????????????????????????????????????????????????????O???_????????A??????????????????????????????????????????????????????????????D??????????????????????@?????????????????????????????????????????????????????????_??????????????????C????????????????????????????????????????????????????????@???????????????????G?????????????????????????????????????????????????????O??@???????????????????G?????????????????????????????????????????????????????[???_????????????????????????????????????????????????????????????????????????L???G??????????????????@?????????????????????????????????????????????????????Bw??????????????G???????C???????????????????????????????????????????A?????????????????????????_???????????????????????????????????????????????????G?????????@?????????????????????????????????????????????C?????????????????????@o????????????????????????????????????????????????????????????????????????????@o?????????_????????????????????????????????????????????A??????????????????????W?????????W?????????????????????????????????????????????_?????????????????????G?????????B?????????????????????????????????????????????C??????????????????????O?????????[?????????????????????????????????????????????O?????????????????????F?????????Aw')
T4, r4, B4 = nice_tree_decomposition(G4)
MI4 = max_independent_set(G4, T4, r4, B4)
print(f"--- {(time.time() - start_time)} seconds ---")
(is_independent_set(G4, MI4), len(MI4)) == (True, 171)

--- 0.5259575843811035 seconds ---


True

In [ ]:
# probably takes a lot of time; use Kernel | Interrupt command from the menu above :)
G4.independent_set()